First things first let's initialise our environment.

*Run the cell below to import our libraries and initialise our LUSID client*

In [1]:
# Import LUSID
import lusid.models as models
import lusid_sample_data as import_data

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import printer as prettyprint
import pandas as pd
import uuid
import math

# Authenticate our user and create our API client
client = import_data.authenticate_secrets()

print ('LUSID Environment Initialised')

LUSID Environment Initialised


![Initailise LUSID](img/multiplecurrencies-initialise.gif)

## 1) Set up our Scope

In [2]:
scope_id = str(uuid.uuid4())[:4]
scope_name = 'UK_High_Growth_Equities_Fund'
scope = '{}_{}'.format(scope_name, scope_id)
print (scope)

UK_High_Growth_Equities_Fund_b329


![Scope](img/multiplecurrencies-scope.gif)

## 2) Load Instruments

In [3]:
instrument_universe = pd.read_csv('data/multiplecurrencies-instruments.csv')
instrument_universe.head(n=10)

instrument_name client_internal currency          isin  \
0          Amazon_Nasdaq_AMZN    imd_34634534      USD  US0231351067   
1           Apple_Nasdaq_AAPL    imd_35345345      USD  US0378331005   
2        USTreasury_2.00_2021    imd_34535347      USD  US912828U816   
3       USTreasury_6.875_2025    imd_34534539      USD  US912810EV62   
4    ExpressScripts_NYSE_ESRX    imd_34352311      USD  US30219G1085   
5  TrinityIndustries_NYSE_TRN    imd_34235200      USD  US8965221091   
6              Trex_NYSE_TREX    imd_32423956      USD  US89531P1057   
7               Cigna_NYSE_CI    imd_32452391      USD  US1255091092   
8             Arcosa_NYSE_ACA    imd_23423409      USD  US0396531008   

           figi exchange_code          country_issue            ticker  \
0  BBG000BVPXP1            UN  united_states_america              AMZN   
1  BBG000B9XVV8            UN  united_states_america              AAPL   
2  BBG00FN3B5K8        BERLIN  united_states_america      T 2 12/31/21   
3  BBG000DQQNJ8      NEW YORK  united_states_america  T 6.875 08/15/25   
4  BBG000C16621            UN  united_states_america              ESRX   
5  BBG000BVL406            UN  united_states_america               TRN   
6  BBG000BTGM43            UN  united_states_america              TREX   
7  BBG00KXXK940            UN  united_states_america                CI   
8  BBG00JGMWFQ5            UN  united_states_america               ACA   

  market_sector  security_type  coupon  
0        equity   common_stock     NaN  
1        equity   common_stock     NaN  
2          govt  us_government   2.000  
3          govt  us_government   6.875  
4        equity   common_stock     NaN  
5        equity   common_stock     NaN  
6        equity   common_stock     NaN  
7        equity   common_stock     NaN  
8        equity   common_stock     NaN

In [4]:
# Initialise our dictionary to hold our instrument definitions
instrument_definitions = {}

# Iterate over our instrument universe
for index, instrument in instrument_universe.iterrows():

    # Create our ISIN property using the default ISIN property definition
    isin_property = models.InstrumentProperty(
        key='Instrument/default/Isin',
        value=models.PropertyValue(
            label_value=instrument['isin']))
    
    # Create our ticker property using the default ticker property definition
    ticker_property = models.InstrumentProperty(
        key='Instrument/default/Ticker',
        value=models.PropertyValue(
            label_value=instrument['ticker']))

    # Create the definition for our instrument
    instrument_definitions[instrument['instrument_name']] = models.InstrumentDefinition(
        name=instrument['instrument_name'],
        identifiers={'Figi': instrument['figi']},
        properties=[
            isin_property,
            ticker_property
        ]
    )

# Call LUSID to upsert our instrument defintions
response = client.upsert_instruments(requests=instrument_definitions)

# Pretty print the response from LUSID
prettyprint.instrument_response(response, identifier='Figi')

Instrument Successfully Upserted: TrinityIndustries_NYSE_TRN
Figi ID: BBG000BVL406
LUSID Instrument ID: LUID_XOEVS1VP


Instrument Successfully Upserted: Apple_Nasdaq_AAPL
Figi ID: BBG000B9XVV8
LUSID Instrument ID: LUID_3SCE05CA


Instrument Successfully Upserted: Amazon_Nasdaq_AMZN
Figi ID: BBG000BVPXP1
LUSID Instrument ID: LUID_KRRFR9HP


Instrument Successfully Upserted: Arcosa_NYSE_ACA
Figi ID: BBG00JGMWFQ5
LUSID Instrument ID: LUID_B4F63NZM


Instrument Successfully Upserted: USTreasury_6.875_2025
Figi ID: BBG000DQQNJ8
LUSID Instrument ID: LUID_3BLPIJ0N


Instrument Successfully Upserted: Cigna_NYSE_CI
Figi ID: BBG00KXXK940
LUSID Instrument ID: LUID_QWB8KLXQ


Instrument Successfully Upserted: USTreasury_2.00_2021
Figi ID: BBG00FN3B5K8
LUSID Instrument ID: LUID_0GAGS4P8


Instrument Successfully Upserted: ExpressScripts_NYSE_ESRX
Figi ID: BBG000C16621
LUSID Instrument ID: LUID_X8K7UME5


Instrument Successfully Upserted: Trex_NYSE_TREX
Figi ID: BBG000BTGM43
LUSID Instrument ID: LU

![Instruments](img/multiplecurrencies-instruments.gif)

## 3) Create our Base Fund Portfolio

In [5]:
base_portfolio_name = '{}_base_fund'.format(scope_name)
print (base_portfolio_name)

UK_High_Growth_Equities_Fund_base_fund


In [6]:
# The date our portfolios were first created
portfolio_creation_date = (datetime.now(pytz.UTC) - timedelta(days=365))

# Create the request to add our portfolio
transaction_portfolio_request = models.CreateTransactionPortfolioRequest(
    display_name=base_portfolio_name,
    code=base_portfolio_name,
    base_currency='GBP',
    description='The portfolio to hold our base fund',
    created=portfolio_creation_date)

# Call LUSID to create our portfolio
portfolio_response = client.create_portfolio(
    scope=scope,
    create_request=transaction_portfolio_request)

# Pretty print the response from LUSID
prettyprint.portfolio_response(portfolio_response)

Portfolio Created
Scope: UK_High_Growth_Equities_Fund_b329
Code: UK_High_Growth_Equities_Fund_base_fund
Portfolio Effective From: 2018-03-12 14:55:41.235602+00:00
Portfolio Created On: 2019-03-12 14:55:44.407654+00:00



![Base-Fund](img/multiplecurrencies-basefundportfolio.gif)

## 4) Set our Holdings

In [7]:
#Import our holdings
holdings = pd.read_csv('data/multiplecurrencies-holdings.csv')
holdings.head()

instrument_name client_internal          isin          figi  \
0     Amazon_Nasdaq_AMZN    imd_34634534  US0231351067  BBG000BVPXP1   
1      Apple_Nasdaq_AAPL    imd_35345345  US0378331005  BBG000B9XVV8   
2   USTreasury_2.00_2021    imd_34535347  US912828U816  BBG00FN3B5K8   
3  USTreasury_6.875_2025    imd_34534539  US912810EV62  BBG000DQQNJ8   

   quantity  unit_cost   total_cost currency  
0      5000    1550.00   7750000.00      USD  
1     49567     190.00   9417730.00      USD  
2    121543      99.25  12063142.75      USD  
3     98444     140.98  13878635.12      USD

In [8]:
holding_adjustments = []

holdings_effective_date = datetime.now(pytz.UTC) - timedelta(days=7)

for row, holding in holdings.iterrows():

    holding_adjustments.append(
        models.AdjustHoldingRequest(
            instrument_identifiers={
                    'Instrument/default/Figi': holding['figi']},
                tax_lots=[
                    models.TargetTaxLotRequest(
                        units=holding['quantity'],
                        cost=models.CurrencyAndAmount(
                            amount=holding['total_cost'],
                            currency=holding['currency']),
                        portfolio_cost=holding['total_cost'],
                        price=holding['unit_cost'])
                ]
        )
    )
    
# Call LUSID to set our holdings
response = client.set_holdings(
    scope=scope,
    code=base_portfolio_name,
    effective_at=holdings_effective_date,
    holding_adjustments=holding_adjustments)

prettyprint.set_holdings_response(response, scope, base_portfolio_name)

Holdings Successfully Set for Portfolio
Scope: UK_High_Growth_Equities_Fund_b329
Code: UK_High_Growth_Equities_Fund_base_fund
Holdings Effective From: 2019-03-05 14:55:46.107586+00:00
Holdings Created On: 2019-03-12 14:55:46.326490+00:00



![Base-Fund-Hodlings](img/multiplecurrencies-basefundportfolioholdings.gif)

## 5) Add Corporate Actions

In [9]:
corporate_action_source_id = '{}_corporate_action_source'.format(base_portfolio_name)

corporate_action_request = models.CreateCorporateActionSourceRequest(
    scope=scope,
    code=corporate_action_source_id)

response = client.create_corporate_action_source(
    request=corporate_action_request)
    
prettyprint.corporate_action_response(response)

Corporate Actions Source Successfully Created
Scope: UK_High_Growth_Equities_Fund_b329
Code: UK_High_Growth_Equities_Fund_base_fund_corporate_action_source


![Corporate-Action-Source](img/multiplecurrencies-corporateactionsource.gif)

In [10]:
corporate_actions = pd.read_csv('data/corporateactions.csv')
corporate_actions.head()

code action_description                             description  \
0  5943592342           dividend                          Apple Dividend   
1  5943592343             merger          Cigna & Express Scripts Merger   
2  5943592343             merger          Cigna & Express Scripts Merger   
3  5943592347           spin-off  Acosa spin-off from Trinity Industries   
4  5943592498              split                Trex 2 for 1 stock split   

  announcement_date     ex_date record_date payment_date  \
0        29/01/2019  08/02/2019  11/02/2019   14/02/2019   
1        08/03/2018  10/12/2018  13/12/2018   20/12/2018   
2        08/03/2018  10/12/2018  13/12/2018   20/12/2018   
3        12/12/2017  17/10/2018  20/10/2018   01/11/2018   
4        07/05/2018  21/05/2018  23/05/2018   19/06/2018   

  input_instrument_figi instrument_input_ticker        input_instrument_name  \
0          BBG000B9XVV8                    AAPL                    APPLE INC   
1          BBG000C16621                    ESRX  EXPRESS SCRIPTS HOLDINGS CO   
2          BBG000C16621                    ESRX  EXPRESS SCRIPTS HOLDINGS CO   
3          BBG000BVL406                     TRN       TRINITY INDUSTRIES INC   
4          BBG000BTGM43                    TREX             TREX COMPANY INC   

   input_units_factor  input_cost_factor output_instrument_figi  \
0                   1                  1                    NaN   
1                   1                  1                    NaN   
2                   1                  1           BBG00KXXK940   
3                   3                  1           BBG00JGMWFQ5   
4                   1                  1           BBG000BTGM43   

  output_instrument_name output_ticker output_instrument_internal  \
0                    NaN           NaN                    CCY_GBP   
1                    NaN           NaN                    CCY_USD   
2             CIGNA CORP            CI                        NaN   
3             ARCOSA INC           ACA                        NaN   
4       TREX COMPANY INC          TREX                        NaN   

   output_units_factor  output_cost_factor  dividend_yield  
0               0.0170              0.0000           0.017  
1              48.7500              0.0000             NaN  
2               0.2434              1.9337             NaN  
3               1.0000              1.0324             NaN  
4               2.0000              2.0000             NaN

In [15]:
actions = {}


for row, action in corporate_actions.iterrows():
    
    actions[action['code']] = {}
    
    luid = client.get_instrument(
        identifier_type='Figi',
        identifier=action['input_instrument_figi']).lusid_instrument_id
    
    actions[action['code']].setdefault(
        'input_transition', models.CorporateActionTransitionComponent(
            instrument_uid=luid,
            units_factor=action['input_units_factor'],
            cost_factor=action['input_cost_factor']))
    
    actions[action['code']].setdefault(
        'output_transitions', []).append(
            models.CorporateActionTransitionComponent(
                instrument_uid=luid,
                units_factor=action['input_units_factor'],
                cost_factor=action['input_cost_factor']))
    
actions_list = []

for action_id, action in actions.items():
    
    transition = models.CorporateActionTransition(
        input_transition=actions[action_id]['input_transition'],
        output_transitions=actions[action_id]['output_transitions']
    )
    
    actions_list.append(
        models.CreateCorporateAction(
            corporate_action_code=action_id,
            announcement_date=datetime.now(pytz.UTC)+timedelta(days=1),
            ex_date=datetime.now(pytz.UTC)+timedelta(days=1),
            record_date=datetime.now(pytz.UTC)+timedelta(days=1),
            payment_date=datetime.now(pytz.UTC)+timedelta(days=1),
            transitions=[transition])
    )

response = client.batch_upsert_corporate_actions(
    scope=scope,
    code=corporate_action_source_id,
    actions=actions_list)

prettyprint.corporate_action_upsert_response(response)

ErrorResponseException: The supplied collection contains one or more duplicate items - which is not permitted. Duplicated ids in items.

In [ ]:
response = client.upsert_portfolio_details(
    scope=scope,
    code=base_portfolio_name,
    details=models.CreatePortfolioDetails(
        corporate_action_source_id=models.ResourceId(
            scope=scope,
            code=corporate_action_source_id),
        base_currency='GBP'))

print (response)

![Corporate-Actions](img/multiplecurrencies-corporateactions.gif)

## 6) Value our Base Fund Portfolio

### a) Analytic store

In [ ]:
prices = pd.read_csv('data/multiplecurrencies-prices.csv')
prices.head()

In [ ]:
quote_date = holdings_effective_date

# Create analytics store request
analytics_store_request = models.CreateAnalyticStoreRequest(
    scope=scope,
    date_property=quote_date)

# Call LUSID to create our analytics store
client.create_analytic_store(request=analytics_store_request)

instrument_analytics = []

for row, quote in prices.iterrows():

    luid = client.get_instrument(
        identifier_type='Figi',
        identifier=quote['instrument_figi']).lusid_instrument_id

    instrument_analytics.append(
        models.InstrumentAnalytic(
            instrument_uid=luid,
            value=quote['price']))

response = client.set_analytics(
    scope=scope,
    year=quote_date.year,
    month=quote_date.month,
    day=quote_date.day,
    data=instrument_analytics)

print (response.key)


![Analytic-Stores](img/multiplecurrencies-analyticstores.gif)

### b) Aggregation

In [ ]:
aggregation_request = models.AggregationRequest(
    recipe_id=models.ResourceId(
        scope=scope,
        code='default'),
        effective_at=holdings_effective_date,
        metrics=[
            models.AggregateSpec(key='Holding/default/SubHoldingKey',
            op='Value'),
            models.AggregateSpec(key='Instrument/default/Name',
            op='Value'),
            models.AggregateSpec(key='Holding/default/Units',
            op='sum'),
            models.AggregateSpec(key='Holding/default/Cost',
            op='sum'),
            models.AggregateSpec(key='Holding/default/PV',
            op='sum'),
        ],
        group_by=[
            'Holding/default/SubHoldingKey'
        ])

response = client.get_aggregation_by_portfolio(
    scope=scope,
    code=base_portfolio_name,
    request=aggregation_request)

print(response.data)

![Analytic-Stores](img/multiplecurrencies-aggregatedbase.gif)

## 7) Securitise our Base Fund Portfolio

### a) Securitiese with x units

In [ ]:
# Create our request to define a new property
property_request = models.CreatePropertyDefinitionRequest(
    domain='Instrument',
    scope=scope,
    code='total_circulation',
    value_required=False,
    display_name='total_circulation',
    data_type_id=models.ResourceId(scope='default', code='number'))

# Call LUSID to create our new property
response = client.create_property_definition(definition=property_request)

# Grab the key off the response to use when referencing this property in other LUSID calls
circulation_property_key = response.key

# Pretty print our key
prettyprint.heading('Circulation Property Key', circulation_property_key)

![Analytic-Stores](img/multiplecurrencies-circulationproperty.gif)

In [ ]:
instrument_definition = models.InstrumentDefinition(
    name=base_portfolio_name,
    identifiers={'ClientInternal': base_portfolio_name},
    properties=[models.InstrumentProperty(
        key=circulation_property_key,
        value=models.PropertyValue(
            metric_value=models.MetricValue(
                value=40000.00
                )))],
    look_through_portfolio_id=models.ResourceId(
        scope=scope,
        code=base_portfolio_name))

response = client.upsert_instruments(requests={'look-through':instrument_definition}
    )

print (response.values)

![Analytic-Stores](img/multiplecurrencies-securitisebase.gif)

### b) Calculalate price from aggregation and store in the aggregation store

In [ ]:
aggregation_request = models.AggregationRequest(
    recipe_id=models.ResourceId(
        scope=scope,
        code='default'),
        effective_at=holdings_effective_date,
        metrics=[
            models.AggregateSpec(key='Holding/default/Cost',
            op='sum'),
            models.AggregateSpec(key='Holding/default/PV',
            op='sum'),
        ],
        group_by=[
            'Portfolio/default/Name'
        ])

response = client.get_aggregation_by_portfolio(
    scope=scope,
    code=base_portfolio_name,
    request=aggregation_request)

print (response.data)

total_value = response.data[0]['Sum(Holding/default/PV)']
total_cost = response.data[0]['Sum(Holding/default/Cost)']

In [ ]:
instrument_analytics = []

quote_date = holdings_effective_date

base_fund_instrument = client.get_instrument(
    identifier_type='ClientInternal',
    identifier=base_portfolio_name,
    instrument_property_keys=[circulation_property_key])

luid = base_fund_instrument.lusid_instrument_id
total_circulation = base_fund_instrument.properties[0].value
price = total_value / total_circulation

print (price)

instrument_analytics.append(
    models.InstrumentAnalytic(
        instrument_uid=luid,
        value=price))

response = client.set_analytics(
        scope=scope,
        year=quote_date.year,
        month=quote_date.month,
        day=quote_date.day,
        data=instrument_analytics)

#print (response.key)

![Base-Fund-Price](img/multiplecurrencies-basefundprice.gif)

## 8) Create Foreign Currency Portfolios

In [ ]:
fund_currencies = ['AUD', 'GBP', 'USD', 'EUR', 'JPY']

for currency in fund_currencies:
    
    portfolio_name = '{}_{}'.format(base_portfolio_name, currency)
    
    # Create the request to add our portfolio
    transaction_portfolio_request = models.CreateTransactionPortfolioRequest(
        display_name=portfolio_name,
        code=portfolio_name,
        base_currency=currency,
        description='The portfolio to hold our base fund in currency {}'.format(currency),
        created=portfolio_creation_date)

    # Call LUSID to create our portfolio
    portfolio_response = client.create_portfolio(
        scope=scope,
        create_request=transaction_portfolio_request)

    # Pretty print the response from LUSID
    prettyprint.portfolio_response(portfolio_response)

![Foreign-Currency-Portfolios](img/multiplecurrencies-foreigncurrencyportfolios.gif)

## 9) Add Base Fund Units

In [ ]:
fx_rates = pd.read_csv('data/multiplecurrencies-fxrates.csv')
fx_rates.head()

In [ ]:
quote_requests = []

for index, fx_rate in fx_rates.iterrows():
    
    quote_requests.append(
        models.UpsertQuoteRequest(
            quote_id=models.QuoteId(
                instrument_id=fx_rate['pair'],
                instrument_id_type='CurrencyPair',
                quote_type='Rate',
                price_side='Mid'),
            metric_value=models.MetricValue(
                value=fx_rate['rate'],
                unit='rate'),
            quote_lineage=models.QuoteLineage(
                data_vendor='system',
                contributor='default'),
            effective_at=holdings_effective_date))
    
response = client.upsert_quotes(
    scope=scope,
    quotes=quote_requests)

print (response)

In [ ]:
fx_rates = {}

for currency in fund_currencies:
    if currency == 'USD':
        fx_rates['USD'] = 1
        continue
        
    response = client.get_quotes(
        scope=scope,
        quote_ids=[
            models.QuoteId(
                instrument_id='{}/USD'.format(currency),
                instrument_id_type='CurrencyPair',
                quote_type='Rate',
                price_side='Mid')
        ],
        effective_at=holdings_effective_date,
        as_at=None,
        max_age=None,
        page=None,
        limit=None)
    
    fx_rates[currency] = response.found[0].metric_value.value

print (fx_rates)

In [ ]:
base_fund_units_per_portfolio = total_circulation/len(fund_currencies)
total_cost_per_portfolio = total_cost/len(fund_currencies)

for currency in fund_currencies:
        
    holding_adjustments = []
    holding_adjustments.append(
        models.AdjustHoldingRequest(
            instrument_identifiers={
                    'Instrument/default/ClientInternal': base_portfolio_name},
                tax_lots=[
                    models.TargetTaxLotRequest(
                        units=base_fund_units_per_portfolio,
                        cost=models.CurrencyAndAmount(
                            amount=total_cost_per_portfolio,
                            currency='USD'),
                        portfolio_cost=total_cost_per_portfolio/fx_rates[currency], 
                        price=total_cost/total_circulation)
                ]
        )
    )

    # Call LUSID to set our holdings
    response = client.set_holdings(
        scope=scope,
        code='{}_{}'.format(base_portfolio_name, currency),
        effective_at=holdings_effective_date,
        holding_adjustments=holding_adjustments)

    prettyprint.set_holdings_response(response, scope, '{}_{}'.format(base_portfolio_name, currency))

![Foreign-Currency-Portfolios](img/multiplecurrencies-foreigncurrencyportfoliosholdbase.gif)

## 10) Add Hedging Transactions

In [ ]:
# To remove later
instrument_definition = models.InstrumentDefinition(
    name=base_portfolio_name,
    identifiers={'ClientInternal': base_portfolio_name},
    properties=[models.InstrumentProperty(
        key=circulation_property_key,
        value=models.PropertyValue(
            metric_value=models.MetricValue(
                value=40000.00
                )))])

response = client.upsert_instruments(requests={'look-through':instrument_definition}
    )

In [ ]:
for currency in fund_currencies:
    if currency == 'USD':
        continue 
        
    exchange_rate = models.PerpetualPropertyValue(
        metric_value=models.MetricValue(
            value=1/fx_rates[currency],
            unit='fx_rate'))
        
    hedge_transactions = []
    
    hedge_transactions.append(
        models.TransactionRequest(
            transaction_id='tid_'.format(uuid.uuid4()),
            type='FwdFxSell',
            instrument_identifiers={
                'Instrument/default/Currency': 'USD'},
            transaction_date=holdings_effective_date,
            settlement_date=holdings_effective_date+timedelta(days=7),
            units=round(total_cost_per_portfolio,0),
            transaction_price=models.TransactionPrice(
                price=1,
                type='Price'),
            total_consideration=models.CurrencyAndAmount(
                amount=total_cost_per_portfolio / fx_rates[currency],
                currency=currency),
            exchange_rate=1/fx_rates[currency],
            transaction_currency='USD',
            source='Client'
            #properties={'Trade/default/TradeToPortfolioRate': exchange_rate}
        ))
    
    response = client.upsert_transactions(
        scope=scope,
        code='{}_{}'.format(base_portfolio_name, currency),
        transactions=hedge_transactions)
            
    prettyprint.transactions_response(
        response, 
        scope, 
        '{}_{}'.format(base_portfolio_name, currency))

## 11) Value Currency Portfolios

In [ ]:
prices = pd.read_csv('data/multiplecurrencies-prices2.csv')
prices.head()

In [ ]:
quote_date = holdings_effective_date+timedelta(days=5)

# Create analytics store request
analytics_store_request = models.CreateAnalyticStoreRequest(
    scope=scope,
    date_property=quote_date)

# Call LUSID to create our analytics store
client.create_analytic_store(request=analytics_store_request)

instrument_analytics = []

for row, quote in prices.iterrows():

    luid = client.get_instrument(
        identifier_type='Figi',
        identifier=quote['instrument_figi']).lusid_instrument_id

    instrument_analytics.append(
        models.InstrumentAnalytic(
            instrument_uid=luid,
            value=quote['price']))

response = client.set_analytics(
    scope=scope,
    year=quote_date.year,
    month=quote_date.month,
    day=quote_date.day,
    data=instrument_analytics)

print (response.key)

In [ ]:
aggregation_request = models.AggregationRequest(
    recipe_id=models.ResourceId(
        scope=scope,
        code='default'),
        effective_at=quote_date,
        metrics=[
            models.AggregateSpec(key='Holding/default/PV',
            op='sum'),
        ],
        group_by=[
            'Portfolio/default/Name'
        ])

response = client.get_aggregation_by_portfolio(
    scope=scope,
    code=base_portfolio_name,
    request=aggregation_request)

print (response.data)

total_value = response.data[0]['Sum(Holding/default/PV)']

In [ ]:
instrument_analytics = []

base_fund_instrument = client.get_instrument(
    identifier_type='ClientInternal',
    identifier=base_portfolio_name,
    instrument_property_keys=[circulation_property_key])

luid = base_fund_instrument.lusid_instrument_id
price = total_value / total_circulation

print (price)

instrument_analytics.append(
    models.InstrumentAnalytic(
        instrument_uid=luid,
        value=price))

response = client.set_analytics(
        scope=scope,
        year=quote_date.year,
        month=quote_date.month,
        day=quote_date.day,
        data=instrument_analytics)

#print (response.key)

In [ ]:
interest_rates = pd.read_csv('data/multiplecurrencies-interestrates.csv')
interest_rates.head()

In [ ]:
fx_rates = pd.read_csv('data/multiplecurrencies-fxrates2.csv')
fx_rates.head()

In [ ]:
quote_requests = []

for index, fx_rate in fx_rates.iterrows():
    
    quote_requests.append(
        models.UpsertQuoteRequest(
            quote_id=models.QuoteId(
                instrument_id=fx_rate['pair'],
                instrument_id_type='CurrencyPair',
                quote_type='Rate',
                price_side='Mid'),
            metric_value=models.MetricValue(
                value=fx_rate['rate'],
                unit='rate'),
            quote_lineage=models.QuoteLineage(
                data_vendor='system',
                contributor='default'),
            effective_at=holdings_effective_date))
    
response = client.upsert_quotes(
    scope=scope,
    quotes=quote_requests)

print (response)

In [ ]:
fx_rates = {}

for currency in fund_currencies:
    if currency == 'USD':
        fx_rates['USD'] = 1
        continue
        
    response = client.get_quotes(
        scope=scope,
        quote_ids=[
            models.QuoteId(
                instrument_id='{}/USD'.format(currency),
                instrument_id_type='CurrencyPair',
                quote_type='Rate',
                price_side='Mid')
        ],
        effective_at=holdings_effective_date,
        as_at=None,
        max_age=None,
        page=None,
        limit=None)
    
    fx_rates[currency] = response.found[0].metric_value.value

print (fx_rates)

In [ ]:
for currency in fund_currencies:
    
    response = client.get_transactions(
        scope=scope,
        code='{}_{}'.format(base_portfolio_name, currency),
        from_transaction_date=holdings_effective_date,
        to_transaction_date=quote_date,
        as_at=None,
        sort_by=None,
        start=None,
        limit=None,
        instrument_property_keys=None,
        filter="type eq 'FwdFxSell'")
    
    if len(response.values) == 0:
        continue
        
    hedge_transaction = response.values[0]
    
    print (hedge_transaction)
    
    spot_rate = 1/fx_rates[currency]
    interest_rate_1 = interest_rates.loc[interest_rates['currency'] == 'USD', 'interest_rate'].values[0]
    interest_rate_2 = interest_rates.loc[interest_rates['currency'] == currency, 'interest_rate'].values[0]
    time = (hedge_transaction.settlement_date - quote_date).days / 365.2425
    print ('Spot rate: ', spot_rate)
    print ('IR 1: ', interest_rate_1)
    print ('IR 2: ', interest_rate_2)
    print ('Time: ', time)
    
    forward_price = spot_rate * math.exp((interest_rate_2 - interest_rate_1)*time)
    print ('Forward Price: ', forward_price)
    strike_price = hedge_transaction.exchange_rate
    print ('Strike Price: ',  strike_price)
    current_value = (forward_price - strike_price) * math.exp(-interest_rate_2*time)
    print ('Current Value: ', current_value)
    
    

In [ ]:
USDGBP_fx_forward_price = 1/fx_rates['GBP']*math.exp((interest_rates['GBP']-interest_rates['USD'])*

## 12) Securitise Currency Portfolios

## 13) Create Share Class Portfolios

## 15) Add Currency Fund Holdings

## 16) Value Each Share Class

## 17) Add transactions to end portfolio

## 18) Value each portfolio (bubble up)